In [ ]:
# default_exp dlc_importer

# DLCImporter

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
import pandas as pd

class DLCImporter:
    """Used to import DLC result files."""
    
    def import_hdf(self, file):
        df = pd.read_hdf(file)
        df.columns = df.columns.droplevel(0) # drop redundant scorer
        columns_to_drop = ['a1', 'a2', 'a3', 'a4', 'b1', 'b2', 'b3', 'b4']
        df = df.drop(columns_to_drop, axis=1, level=0)
        df.columns = df.columns.remove_unused_levels()
        return df

`DLCImporter` can be used to import result files from DeepLabCut in H5 format.
It will automatically drop the unnecessary `scorer` level.

In [ ]:
imp = DLCImporter()
df = imp.import_hdf('example_data/coordinates.h5')
display(df)

bodyparts        head                               beak              \
coords              x           y likelihood           x           y   
0          773.376465  231.518768   0.999999  726.495178  235.638046   
1          773.129822  231.487213   0.999999  725.662231  235.242844   
2          773.009827  231.793518   0.999999  726.025696  235.272522   
3          773.748779  231.791260   0.999999  726.288940  235.864319   
4          774.934326  231.623734   0.999999  726.298279  235.749908   
..                ...         ...        ...         ...         ...   
95         691.788513  232.490265   1.000000  673.796082  238.801743   
96         691.545410  232.707413   1.000000  673.634888  238.658234   
97         691.117371  232.242767   1.000000  673.748840  239.055954   
98         691.294067  232.225220   1.000000  673.927002  239.141891   
99         691.483643  232.269226   1.000000  673.797241  239.390625   

bodyparts              left_neck                         right_neck  ...  \
coords    likelihood           x           y likelihood           x  ...   
0           0.999981  726.502014  277.634125   0.999998  803.271179  ...   
1           0.999951  725.964478  278.003082   0.999999  803.197144  ...   
2           0.999978  725.764893  278.884918   0.999998  802.567810  ...   
3           0.999985  725.889465  279.045715   0.999998  803.356934  ...   
4           0.999990  726.302551  278.388367   0.999999  802.530273  ...   
..               ...         ...         ...        ...         ...  ...   
95          0.018886  697.399841  282.134796   0.999998  737.725342  ...   
96          0.016135  697.256165  283.058899   0.999999  736.505920  ...   
97          0.007289  696.269043  282.351929   0.999999  735.976685  ...   
98          0.004682  695.629456  282.407013   1.000000  735.639404  ...   
99          0.010126  695.367371  281.720947   0.999999  735.199585  ...   

bodyparts right_middle_wing right_down_wing                         \
coords           likelihood               x           y likelihood   
0                  0.999998      866.702393  446.583923   0.999997   
1                  0.999997      866.877441  446.645325   0.999989   
2                  0.999997      867.120056  447.921356   0.999995   
3                  0.999992      866.839966  448.009460   0.999994   
4                  0.999995      866.429382  446.349670   0.999998   
..                      ...             ...         ...        ...   
95                 0.999993      866.965027  433.505768   0.999980   
96                 0.999995      868.164307  432.291901   0.999943   
97                 0.999984      868.530457  434.697205   0.999916   
98                 0.999977      868.368958  434.125732   0.999876   
99                 0.999970      868.662292  434.387238   0.999786   

bodyparts        body                               tail              \
coords              x           y likelihood           x           y   
0          804.008545  350.669586   0.999992  874.878601  485.749908   
1          802.684265  345.021454   0.999873  875.375854  487.185547   
2          801.531067  349.937347   0.999946  876.269714  485.816010   
3          802.792908  350.675842   0.999970  875.973022  485.560150   
4          803.659973  351.269745   0.999938  876.481873  485.140839   
..                ...         ...        ...         ...         ...   
95         788.017456  337.912994   0.999999  882.997253  483.786896   
96         788.334045  339.911743   0.999999  884.470215  483.485382   
97         785.626465  338.561829   0.999997  885.270691  485.053131   
98         786.011963  338.520691   0.999997  885.585388  484.755859   
99         785.282776  338.077087   0.999992  885.361023  483.480896   

bodyparts             
coords    likelihood  
0           0.999999  
1           0.999997  
2           0.999999  
3           0.999998  
4           0.999998  
..               ...  
95          1.000000  
96          1.00000

For further processing and analyzes, it can be useful to transform the tracked coordinates into relative coordinates (being relative to a specified bodypart). This can be done by specifying a bodypart as the relative origin.


In [ ]:
#export
def transform_to_relative(df, bodypart):
    df_rel = df.copy()

    for bp in df.columns.levels[0]:
        df_rel[bp, 'x'] = df[bp, 'x'] - df[bodypart, 'x']
        df_rel[bp, 'y'] = df[bp, 'y'] - df[bodypart, 'y']

    return df_rel

In [ ]:
df_relative = transform_to_relative(df, 'body')

test_close(df_relative['head','x'][0], -30.6, 0.1)
test_close(df_relative['head','y'][0], -119.1, 0.1)
test_close(df_relative['tail','x'][0], 70.87, 0.1)
test_close(df_relative['tail','y'][0], 135.08, 0.1)

By using the `arctan2` function of Numpy, we are able to calculate the angle of the vector from the origin to a specific point.

In [ ]:
import numpy as np

y = np.array([1, 1, 1, -1, 0.75])
x = np.array([0, 1, -1, 1, 1.5])

result = np.degrees(np.arctan2(y, x))
expected = np.array([90, 45, 135, -45, 26.565051])

np.testing.assert_almost_equal(result, expected, decimal=3)

In [ ]:
# export
def add_middle_neck(df):
    df_middle_neck = df.copy()
    df_middle_neck['middle_neck', 'x'] = (df_middle_neck['left_neck','x'] + df_middle_neck['right_neck', 'x']) / 2
    df_middle_neck['middle_neck', 'y'] = (df_middle_neck['left_neck','y'] + df_middle_neck['right_neck', 'y']) / 2
    return df_middle_neck

Since we already moved the body into the origin, we can now decide for a good point for determining the angle of the pigeon rotation.
The vector between the body and middle point between left_neck and right_neck seems to be suitable.
Therefore, we first calculate this point, we can later use to determine the rotation of the pigeon.

In [ ]:
# assume we have pandas Dataframe
df_example = df.copy()
df_example['left_neck', 'x'][0] = 1
df_example['left_neck', 'y'][0] = 1
df_example['right_neck', 'x'][0] = 2
df_example['right_neck', 'y'][0] = 0.5

df_result = add_middle_neck(df_example)

test_close(df_result['middle_neck','x'][0], 1.5, 0.1)
test_close(df_result['middle_neck','y'][0], 0.75, 0.1)


Based on the `middle_neck` coordinates, we now add the corresponding rotation to the dataframe.

In [ ]:
# export
import numpy as np

def add_rotation(df):
    df = df.copy()
    df['rotation_angle'] = np.degrees(np.arctan2(df['middle_neck', 'y'], df['middle_neck', 'x']))
    return df

In [ ]:
df_result = add_rotation(df_result)
test_close(df_result['rotation_angle'][0], 26.565051, 0.1)

In [ ]:
# export
def apply_rotation(df):
    df = df.copy()
    df.apply(_rotate_row, axis=1)
    return df

def _rotate_row(x):
    theta = np.radians(x['rotation_angle'][0] - 90)

    body_parts = list(x.index.levels[0])
    body_parts.remove('rotation_angle')

    for b in body_parts:
        x1, y = x[b,'x'], x[b,'y']

        c, s = np.cos(theta), np.sin(theta)
        rot = np.matrix([[c, s], [-s, c]])

        rotated = np.dot(rot, [x1, y])

        x[b, 'x'] = rotated[0, 0]
        x[b, 'y'] = rotated[0, 1]

    return x

Finally, we can rotate all relative coordinates by this angle around the origin (which is the `body`), so we stabilize (and normalize) the pigeon orientation.

In [ ]:
df_example = df_result.copy()
df_rotation_applied = apply_rotation(df_example)

test_close(df_rotation_applied['middle_neck','x'][0], 0, 0.1)
test_close(df_rotation_applied['middle_neck','y'][0], 1.6, 0.1)
test_close(df_rotation_applied['left_neck','x'][0], -0.5, 0.1)
test_close(df_rotation_applied['left_neck','y'][0], 1.34, 0.1)

In [ ]:
df_rotation_applied

bodyparts        head                               beak              \
coords              x           y likelihood           x           y   
0          138.787788  795.267280   0.999999  114.137445  755.177579   
1           39.834331  806.057862   0.999999   20.450851  762.565822   
2           39.749968  806.034966   0.999999   20.775538  762.912084   
3           40.164570  806.722486   0.999999   20.462394  763.353714   
4           39.764190  807.831401   0.999999   19.671357  763.348051   
..                ...         ...        ...         ...         ...   
95          22.820040  729.453436   1.000000   10.640974  714.782625   
96          22.570502  729.299915   1.000000   10.757272  714.580993   
97          22.580007  728.745543   1.000000   10.152300  714.830175   
98          23.107823  728.890986   1.000000   10.575034  715.020711   
99          23.165527  729.082972   1.000000   10.328965  714.985249   

bodyparts             left_neck                        right_neck  ...  \
coords    likelihood          x           y likelihood          x  ...   
0           0.999981  -0.447214    1.341641   0.999998   0.447214  ...   
1           0.999951 -19.756414  777.122783   0.999999  19.756414  ...   
2           0.999978 -20.418605  777.235201   0.999998  20.418605  ...   
3           0.999985 -20.368948  777.410531   0.999998  20.368948  ...   
4           0.999990 -20.530139  777.556424   0.999999  20.530139  ...   
..               ...        ...         ...        ...        ...  ...   
95          0.018886 -21.767582  751.992523   0.999998  21.767582  ...   
96          0.016135 -22.644057  752.180660   0.999999  22.644057  ...   
97          0.007289 -22.610462  751.000638   0.999999  22.610462  ...   
98          0.004682 -22.421310  750.434105   1.000000  22.421310  ...   
99          0.010126 -21.835634  749.950450   0.999999  21.835634  ...   

bodyparts right_down_wing       body                               tail  \
coords         likelihood          x           y likelihood           x   
0                0.999997  45.915139  875.951311   0.999992  -43.210321   
1                0.999989 -57.324910  871.811727   0.999873 -167.074372   
2                0.999995 -62.079260  872.384298   0.999946 -165.185364   
3                0.999994 -62.174409  873.833132   0.999970 -164.840502   
4                0.999998 -63.478050  874.774441   0.999938 -165.442840   
..                    ...        ...         ...        ...         ...   
95               0.999980 -42.519259  856.357878   0.999999 -146.215377   
96               0.999943 -44.237424  857.352676   0.999999 -145.367203   
97               0.999916 -44.239268  854.327772   0.999997 -146.937995   
98               0.999876 -43.539081  854.701944   0.999997 -145.933244   
99               0.999786 -43.338690  853.865864   0.999992 -144.771195   

bodyparts                          middle_neck             rotation_angle  
coords              y likelihood             x           y                 
0          999.749173   0.999999  1.110223e-16    1.677051      26.565051  
1          987.784793   0.999997 -8.526513e-14  811.093066      19.497707  
2          988.220422   0.999999  5.684342e-14  810.740481      19.515082  
3          987.889166   0.999998 -2.842171e-14  811.284220      19.526845  
4          988.033791   0.999998 -2.842171e-14  810.725635      19.459142  
..                ...        ...           ...         ...            ...  
95         996.170153   1.000000  5.684342e-14  765.418580      20.367862  
96         997.453784   1.000000 -2.842171e-14  764.706621      20.370898  
97         998.694128   0.999999  8.526513e-14  763.790083      20.349113  
98         998.976081   0.999999  8.526513e-14  763.444060      20.384479  
99         998.328212   1.000000  8.526513e-14  763.082467      20.387066  

[100 rows x 39 columns]